In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.metric import MetricCalculator
from mesostat.utils.qt_helper import gui_fname, gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
import lib.analysis.pid as pid

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yaro/neuronal-raw-pooled'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
h5outname = 'sych_result_higher_order_df.h5'

In [ ]:
mc = MetricCalculator(serial=False, verbose=False, nCore=4)

# All-Distribution

**TODO**:
* [] Drop shitty sessions
* [] Try composite p-values
* [+] For Info3D, drop PCA1
* [] Consider re-doing analysis with PCA1 dropped
* Fraction Significant triplets per session
    * [+] Do regression on PID instead of Naive vs Expert
    * [ ] Binomial test fraction significant PID's above chance
    * [ ] Test if regression explained by having more iGO trials in expert
* Most significant triplets
    * [ ] Plot pvalue vs performance for top10 sessions

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
pwdAllH5 = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed/sych_result_multiregional_pid_df3.h5'
# pwdAllH5 = '/media/aleksejs/DataHDD/work/data/yaro/pid/sych_result_multiregional_pid_df3.h5'
# pwdAllH5 = gui_fname('h5path', './', '(*.h5)')

In [ ]:
pid.plot_all_results_distribution(dataDB, pwdAllH5, plotstyle='cdf', minTrials=50)

In [ ]:
pid.plot_all_frac_significant_bysession(dataDB, pwdAllH5, minTrials=50)

In [ ]:
# TODO: Linear fit + pval(H0: alpha=0)
pid.plot_all_frac_significant_performance_scatter(dataDB, pwdAllH5, minTrials=50)

TODO:
* Top 10 most synergetic connections
    - Try magnitude vs avg p-value vs fraction significant
    - Plot colorbars to show fraction of sessions by mouse
* Top 10 most synergy-involved regions
    - Count fraction of significant triplets where this region is target

In [ ]:
summaryDF = pid.pid_all_summary_df(pwdAllH5)

In [ ]:
# Precompute fraction of significant sessions for each triplet
pidTypes = ['unique', 'red', 'syn']
mouseSignDict = {}
for keyLabel, dfSession in summaryDF.groupby(['datatype', 'phase']):
    print(keyLabel)
    mouseSignDict[keyLabel] = pid._get_pid_sign_dict(dataDB, keyLabel,
                                                     dfSession.drop(['datatype', 'phase'], axis=1),
                                                     pwdAllH5, pidTypes, minTrials=50, trialType='iGO')

In [ ]:
# 1D projection: Targets that have highest fraction of sessions averaged over sources
for keyLabel, dfSession in summaryDF.groupby(['datatype', 'phase']):
    print(keyLabel)
    
    pid.plot_all_frac_significant_1D_top_n(dataDB, mouseSignDict[keyLabel], '_'.join(keyLabel), pidTypes, nTop=20)

In [ ]:
# 2D projection: Source pairs that have highest fraction of sessions averaged over targets
for keyLabel, dfSession in summaryDF.groupby(['datatype', 'phase']):
    print(keyLabel)
    pid.plot_all_frac_significant_2D_avg(dataDB, mouseSignDict[keyLabel], '_'.join(keyLabel), pidTypes)

In [ ]:
# 3D projection: Triplets with highest fraction of sessions
for keyLabel, dfSession in summaryDF.groupby(['datatype', 'phase']):
    print(keyLabel)
    pid.plot_all_frac_significant_3D_top_n(dataDB, mouseSignDict[keyLabel], '_'.join(keyLabel), pidTypes, nTop=20)

In [ ]:
# Specific 2D projection: Fractions for all source pairs given target
for keyLabel, dfSession in summaryDF.groupby(['datatype', 'phase']):
    print(keyLabel)
    pid.plot_all_frac_significant_2D_by_target(dataDB, mouseSignDict[keyLabel], '_'.join(keyLabel), 'syn', 'VPL',
                                               vmax=1)

In [ ]:
hDict = {
#     "H_ALL"  : ("REW", ['VPL', 'DG_a'], ['VM'])
    "H_ALL"  : ("REW", ['Rt', 'SuG'], ['Cpu'])
#     "H_ALL"  : ("REW", ['Cpu_1', 'VPL'], ['VL'])
#         "H_ALL"  : ("TEX", ['M2', 'S2'], ['VPL'])
}

pid.hypotheses_calc_plot_info3D(dataDB, hDict, #performance='expert',
                                datatypes=['bn_session', 'bn_trial'], trialType='iGO')